# Introduction
This notebook presents the scenario a data scientist working on the creation of a `price prediction model` for the `House Pricing in Belo Horizonte` dataset based on our `Exploratory Data Analysis`.

This is a research phase where `mlflow` is used to manually track experiments and decide on a model to run in `production`.

In [1]:
import logging
from os import getenv

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import mlflow
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.discriminant_analysis import StandardScaler
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import ElasticNet, Lasso, LinearRegression, Ridge
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import FunctionTransformer, Pipeline, make_pipeline

from library.dataset import get_dataset, split_test_dataset
from library.serve import predict
from library.train import train_simple_linear_regression

In [2]:
DATASET_PATH = '../data/data_kaggle_2021.csv'
EXPERIMENT_NAME = 'regression_belo_horizonte_2021'
EXPERIMENT_DESCRIPTION = 'Regression model to predict the price of a real estate property in Belo Horizonte, Brasil '\
    'using the Kaggle Dataset from 2021.'
RANDOM_STATE = 42
TRACKING_SERVER_URI = getenv('TRACKING_SERVER_URI', 'http://localhost:5000')

logging.basicConfig(level=logging.INFO)

## Download the dataset and clean it for training

In [3]:
df, variables = get_dataset(DATASET_PATH)

df.head()

INFO:library.dataset:Reading file ../data/data_kaggle_2021.csv
INFO:library.dataset:The dataset ../data/data_kaggle_2021.csv contains 5981 rows.
INFO:library.dataset:Cleaning the dataset:
INFO:library.dataset:Dropping duplicates...
INFO:library.dataset:   Dropped 0 duplicated rows.
INFO:library.dataset:Standardizing the column names...
INFO:library.dataset:Converting the numeric variables received as text...
INFO:library.dataset:Removing outliers...
INFO:library.dataset:  Removed 793 rows containing outliers.
INFO:library.dataset:Number of rows after cleaning the dataset: 5188


,address,adm_fees,garage_places,price,rooms,square_foot,neighborhood,city,latitude,longitude
0,"Avenida Raja Gabaglia, 1583",470.0,1,330000.0,1,40,Luxemburgo,Belo Horizonte,-19.936415,-43.953396
1,"Rua Espírito Santo, 1171",0.0,1,480000.0,2,55,Centro,Belo Horizonte,-18.864776,-41.121777
2,"Rua dos Expedicionários, 1082",0.0,5,1190000.0,4,411,Santa Amélia,Belo Horizonte,-20.225241,-44.397780
3,"Rua Marechal Hermes, 81",750.0,4,1590000.0,4,158,Gutierrez,Belo Horizonte,-19.937155,-43.958694
4,"Rua Juruna, 110",0.0,6,550000.0,3,278,Pindorama,Belo Horizonte,-19.912982,-44.019944


## Create an experiment in mlflow

In [4]:
mlflow.set_tracking_uri(TRACKING_SERVER_URI)
mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.set_experiment_tag('mlflow.note.content',EXPERIMENT_DESCRIPTION)

## Split our data into train and test datasets
We will save our 20% of our dataset to test and compare the future models.

In [5]:
df_train, df_test = split_test_dataset(df, 0.2, RANDOM_STATE)

INFO:library.dataset:Splitting the dataset
INFO:library.dataset:   Split the dataset into datasets of 4150 and 1038 rows.


## Train our base model
Train a simple linear regression for future reference.

In [7]:
model_uri = train_simple_linear_regression(df_train, variables)

## Add the model to the model registry
We will register our base model in the model registry and give it the `@base` alias. Since it is also our only model, it is also our de facto `@champion`. This way, we will have future challengers try and defeat it.

In [8]:
MODEL_NAME = 'belo_horizonte_price_regression'
registered_model = mlflow.register_model(
    model_uri=model_uri,
    name=MODEL_NAME,
    tags={'mlflow.user': 'Quentin El Guay'}
)

client = mlflow.MlflowClient()
client.set_registered_model_alias(registered_model.name, 'base', registered_model.version)
client.set_registered_model_alias(registered_model.name, 'champion', registered_model.version)

Registered model 'belo_horizonte_price_regression' already exists. Creating a new version of this model...
2024/07/31 21:53:30 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: belo_horizonte_price_regression, version 2
Created version '2' of model 'belo_horizonte_price_regression'.


In [18]:
# MODEL_NAME = 'belo_horizonte_price_regression'
# champion_version = mlflow.pyfunc.load_model(f'models:/{MODEL_NAME}@champion')
# y_pred = champion_version.predict(df_test)

inputs = {
        'adm_fees': '',
        'neighborhood': 'Miramar',
        'square_foot': '79',
        'rooms': '2',
        'garage_places': '--',
    }

# df.head()

# champion_rmse = root_mean_squared_error(df_test[variables['target']], y_pred)
# print(f'Champion\'s rmse on test dataset: {champion_rmse}.')

  adm_fees neighborhood square_foot rooms garage_places
0               Miramar          79     2            --
                                     address  adm_fees  garage_places  \
3362                       Rua dos Timbiras        0.0              6   
4672  Avenida Professor Mário Werneck, 2394     1180.0              3   
3542                         Rua Maranguape      330.0              2   
472            Rua Bernardo Guimarães, 3115        0.0              2   
1788                     Rua Senador Amaral        0.0              5   
...                                      ...       ...            ...   
45        Rua Tabelião Ferreira de Carvalho      650.0              2   
1717                           Rua Hematita      250.0              2   
4236                          Rua São Lucas        0.0              3   
3679                   Rua dos Agrimensores        0.0              3   
3665                        Rua Níquel, 105        0.0              4   

          p

In [57]:
categorical_features = variables['categorical']
numerical_features = variables['numerical']
target = variables['target']

X_train, X_val, y_train, y_val = train_test_split(
    df_train[categorical_features + numerical_features],
    df_train[target],
    test_size=0.2,
    random_state=42
)


def dataframe_to_dict(df:pd.DataFrame):
    return df.to_dict('records')

def objective(params):
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', Pipeline(steps=[('scaler', StandardScaler())]), numerical_features),
            (
                'cat',
                Pipeline(
                    steps=[
                        ('to_dict', FunctionTransformer(dataframe_to_dict)),  # returns a list of dicts
                        ('vectorizer', DictVectorizer()),  # list of dicts -> feature matrix
                    ]
                ), categorical_features
            )
        ]
    )

    pipeline = make_pipeline(
        preprocessor,
        TransformedTargetRegressor(
            regressor=ElasticNet(**params), func=np.log1p, inverse_func=np.expm1
        ),
    )
    
    with mlflow.start_run(run_name='ElasticNet Optimization', nested=True):
        mlflow.log_params(params)
          
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_val)
        rmse_optimized = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric('rmse', rmse_optimized)
        mlflow.sklearn.log_model(pipeline, 'model')

        return {'loss': rmse_optimized, 'status': STATUS_OK}

search_space = {
    'alpha': hp.uniform('alpha', 0.0001, 0.001),
    'l1_ratio': hp.uniform('x_l1', 0, 0.2),
    'max_iter': scope.int(hp.quniform('max_iter', 100, 1000, 1)),
    'random_state': 42
}

with mlflow.start_run(run_name='Elasticnet'):
    fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=100,
        trials=Trials(),
        rstate=np.random.default_rng(42)
    )


  0%|                                                                                                           | 0/100 [00:00<?, ?trial/s, best loss=?]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.000905 seconds
INFO:hyperopt.tpe:TPE using 0 trials


  1%|▊                                                                                 | 1/100 [00:02<03:23,  2.06s/trial, best loss: 406011.1939152241]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.001066 seconds
INFO:hyperopt.tpe:TPE using 1/1 trials with best loss 406011.193915


  2%|█▋                                                                                 | 2/100 [00:04<03:24,  2.08s/trial, best loss: 403189.564386534]

...

 98%|███████████████████████████████████████████████████████████████████████████████▍ | 98/100 [03:46<00:04,  2.23s/trial, best loss: 399014.4720536462]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.001323 seconds
INFO:hyperopt.tpe:TPE using 98/98 trials with best loss 399014.472054
/home/quentin/.cache/pypoetry/virtualenvs/belo-horizonte-estate-pricing-JU3YA-Le-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.23123027800960472, tolerance: 0.17493331201151205
  model = cd_fast.sparse_enet_coordinate_descent(



 99%|████████████████████████████████████████████████████████████████████████████████▏| 99/100 [03:48<00:02,  2.23s/trial, best loss: 399014.4720536462]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.001045 seconds
INFO:hyperopt.tpe:TPE using 99/99 trials with best loss 399014.472054


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [03:50<00:00,  2.31s/trial, best loss: 399014.4720536462]


In [58]:
logged_model = 'runs:/69352bca71074a0abbebf5fc4d8abf28/model'

# Load model as a PyFuncModel.
challenger_model = mlflow.pyfunc.load_model(logged_model)
y_pred = challenger_model.predict(df_test)
challenger_rmse = root_mean_squared_error(df_test[variables['target']], y_pred)
print(f'Challenger\'s rmse on test dataset: {rmse}.')

Challenger's rmse on test dataset: 417019.35605824785.
